ライブラリのインポート

データの用意

In [1]:
!pip install openai
!pip install tiktoken
!pip install rank_bm25
!pip install janome
!pip install tqdm

import json
import pandas as pd
from openai import OpenAI
from rank_bm25 import BM25Okapi
from janome.tokenizer import Tokenizer
import numpy as np
from tqdm import tqdm

データのデータフレーム変換

In [2]:
file_path = "データセットへのパス"

# JSONLファイルを読み込む
data = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line))

# データをデータフレームに変換
df = pd.DataFrame(data)

# 特定の列の値が特定の文字列に一致する行を抽出
# target_string = '気象庁'
# filtered_df = df[df['copyright'].str.contains(target_string)]
filtered_df = df
# 抽出したデータフレームを表示
len(df)


22794

ベクトル変換

In [3]:
client = OpenAI(api_key="your-api-key")

def get_embedding(text, model="text-embedding-3-large"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [4]:
# 答え用
filtered_df["Answer_emb"] = filtered_df.Answer.apply(lambda x: get_embedding(x, model="text-embedding-3-large"))
# 質問用
filtered_df["Question_emb"] = filtered_df.Question.apply(lambda x: get_embedding(x, model="text-embedding-3-large"))
filtered_df.to_csv("all_emb2.csv")

In [10]:
filtered_df

,Question,Answer,copyright,url
0,公共工事について会計検査院はどのように検査をしているのですか。,検査に当たっては、国が行う工事や地方公共団体が国の補助金で行う工事などについて、工事に関する...,会計検査院,https://www.jbaudit.go.jp/general/faq.html
1,公共工事について会計検査院はどのような指摘をしていますか。,公共工事は、計画、設計、予定価格の積算、契約、施工、しゅん工・引渡し、成果物の利用という一連...,会計検査院,https://www.jbaudit.go.jp/general/faq.html
2,工事の検査を行うには、専門的な知識が必要だと思いますが、工事検査のために技術系の職員を多く採...,技術系の職員も採用していますが、工事の検査には、技術系以外の職員も多く従事しています。\n ...,会計検査院,https://www.jbaudit.go.jp/general/faq.html
3,独立行政法人等はすべて会計検査院の検査対象ですか。,会計検査院の検査対象は、検査院の検査を受けなければならないとされているもの（会計検査院の検査...,会計検査院,https://www.jbaudit.go.jp/general/faq.html
4,独立行政法人等に対して、どのような検査を行っているのですか。,国の決算を検査する場合も同じですが、その法人の会計について、「正確性、合規性、経済性、効率性...,会計検査院,https://www.jbaudit.go.jp/general/faq.html
...,...,...,...,...
22789,ゼロシナリオ以外のシナリオでは、原子力発電に関する事故の補償、除染、廃炉、核廃棄物の処理など...,原子力発電の発電コストには、原子力発電に関する事故の補償、除染、廃炉、核廃棄物の処理などに要...,内閣官房,https://www.cas.go.jp/jp/seisaku/npu/policy09/...
22790,日本は、諸外国に比して何倍もの値段で天然ガスを購入していると聞いたことがあります。より安価な...,３つのシナリオの経済影響分析等においては、化石燃料の価格については、現在の輸入価格とＩＥＡ（...,内閣官房,https://www.cas.go.jp/jp/seisaku/npu/policy09/...
22791,意見聴取会は、なぜ１１都市のみの開催だったのですか。もっとたくさんの地域で開催すべきではない...,意見聴取会は、全国主要１０都市及び福島市で開催しました。全国でブロックごとにバランスよく配し...,内閣官房,https://www.cas.go.jp/jp/seisaku/npu/policy09/...
22792,討論型世論調査とは何ですか。今回、なぜ討論型世論調査を実施することとしたのですか。,討論型世論調査とは、国民の中から無作為で選ばれた皆様に、知識の習得や討議を通じて、課題につい...,内閣官房,https://www.cas.go.jp/jp/seisaku/npu/policy09/...


ベクトル検索評価

In [3]:
#ベクトルデータ読み込み
df_load = pd.read_csv("kishocho_emb.csv")
answer=df_load.Answer_emb.apply(eval).apply(np.array)
question=df_load.Question_emb.apply(eval).apply(np.array)
answer.to_numpy().shape

(450,)

In [64]:
topk=1 #上位何個の中に入っているか
count=0

def cos_sim(v1, v2):#コサイン類似度
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

#ベクトル検索精度
for x,q in enumerate(question):
    hokan_list=[]
    for y,a in enumerate(answer):
        cos=cos_sim(q,a)
        hokan_list.append(cos)
    if x in np.argsort(hokan_list)[-topk:][::-1]:
        count+=1
print("ベクトル検索精度:",count/len(question)*100,"%")
    

ベクトル検索精度: 71.33333333333334 %


キーワード検索精度

In [14]:
#テキストデータ読み込み
df_load = pd.read_csv("kishocho_emb.csv")
answer_texts=df_load.Answer.to_list()
question_texts=df_load.Question.to_list()
# answer.to_numpy().shape

In [74]:
topk=5#上位何個の中に入っているか

t=Tokenizer()
count=0

def tokenize(doc):
    return list(t.tokenize(doc, wakati = True))
def tokenize_hinshi(doc):
    filtered_tokens=[]
    for token in t.tokenize(doc):
        # 品詞が「名詞」のトークンのみを追加
        if token.part_of_speech.split(',')[0] in ['名詞','動詞','形容詞']:
            filtered_tokens.append(token.surface)
    return filtered_tokens
def compute_bm25(corpus, query, k1, b):
    bm25 = BM25Okapi(corpus, k1=k1, b=b)
    scores = bm25.get_scores(query)
    return scores

#回答集は先に
corpus = answer_texts
tokenized_corpus = [tokenize_hinshi(doc) for doc in corpus]
# 初期パラメータ
k1 = 1.2
b = 0.75

for num,question_text in enumerate(tqdm(question_texts)):
    #  クエリの定義
    query = question_text

    tokenized_query = tokenize_hinshi(query)

    # BM25スコアの計算
    bm25_scores = compute_bm25(tokenized_corpus, tokenized_query, k1, b)
    if len(bm25_scores)>=topk:
        if num in np.argsort(bm25_scores)[-topk:][::-1]:
            count+=1
    else:
        if num in np.argsort(bm25_scores)[-len(bm25_scores):][::-1]:
            count+=1
print("BM25(名詞,動詞,形容詞)精度: ",count/len(corpus)*100,"%")


100%|██████████| 450/450 [00:03<00:00, 129.53it/s]

BM25(名詞,動詞,形容詞)精度:  84.66666666666667 %
